MSDS 7331 Data Mining 
Lab 2
July 08, 2018
Group names: Travis Deason, Michael Landrum, & Vanessa Torres
            

Wine Reviews
130k wine reviews with variety, location, winery, price and description

https://www.kaggle.com/zynicide/wine-reviews

In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from nltk.stem import *
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime, timedelta, date
%matplotlib inline
plt.style.use('ggplot')

In [4]:
import nltk
nltk.download("stopwords")
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vanessatorres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from nltk.corpus import stopwords
pstemmer = nltk.PorterStemmer()

In [6]:
import os
print(os.listdir("../vanessatorres/desktop/wine-reviews"))

['.DS_Store', '.Rapp.history', 'winemag-data-130k-v2.csv', 'winemag-data-130k-v2.json', 'winemag-data_first150k.csv']


In [7]:
df = pd.read_csv("../vanessatorres/desktop/wine-reviews/winemag-data-130k-v2.csv", encoding='ISO-8859-14')

In [8]:
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",VulkÃ Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin OâKeefe,@kerinokeefe,Nicosia 2013 VulkÃ Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwineÂ,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwineÂ,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
Unnamed: 0               129971 non-null int64
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(2), object(11)
memory usage: 13.9+ MB


In [10]:
for col in df.columns:
    print(col, df[col].unique().size)

Unnamed: 0 129971
country 44
description 119955
designation 37980
points 21
price 391
province 426
region_1 1230
region_2 18
taster_name 20
taster_twitter_handle 16
title 118840
variety 708
winery 16757


Data Preparation 1 (10pts)

Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.

For the purpose of our models we wanted to see if we could classify wines based on the specific region, a sommlier's description and variety. 

In [11]:
#sent_tokenizer = nltk.data.load(df['description']) # i may want this

In [12]:
reviews = df.description

In [13]:
swords = set(stopwords.words('english'))

In [14]:
ps = PorterStemmer()

In [15]:
word_vecs = reviews\
    .apply(lambda x: [re.sub("[\W]", '', i).lower().strip() for i in x.split()])\
    .apply(lambda x: [ps.stem(i) for i in x if i not in swords and len(i) > 3])

In [16]:
word_vecs.sample(10)

46077    [creami, textur, ripe, muscadet, domin, charac...
34232    [easi, gentl, wine, drink, softli, tannic, che...
76682    [rubberi, rustic, nose, start, stalki, gritti,...
23835    [creami, refin, offer, aroma, fragrant, wild, ...
42275    [subtli, earthi, dusti, miner, mingl, press, a...
45230    [ripe, round, wine, touch, oygen, give, extra,...
52999    [soft, rich, wine, offer, dusti, tannin, well,...
90988    [despit, silki, mediumbodi, textur, pinot, tou...
68349    [pinot, nero, open, aroma, berri, rose, petal,...
32015    [deepli, impress, bordeaux, blend, come, small...
Name: description, dtype: object

In [17]:
word_vecs.apply(lambda x: [ps.stem(i) for i in x]).sample(5)

85976     [exot, chardonnay, reminisc, rhãṁne, white, wi...
14624     [ripe, rich, velveti, wine, show, lack, restra...
15924     [stoni, miner, note, accent, fragrant, blossom...
100700    [balanc, ripe, structur, fine, reflect, miner,...
12607     [brand, recentlycr, appel, part, beaujolai, wi...
Name: description, dtype: object

In [18]:
from collections import Counter

In [19]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vanessatorres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
reviews.shape

(129971,)

In [21]:
df['taster_name'].fillna('none').value_counts(1)

none                  0.201922
Roger Voss            0.196305
Michael Schachner     0.116441
Kerin OâKeefe       0.082911
Virginie Boone        0.073378
Paul Gregutt          0.073339
Matt Kettmann         0.048719
Joe Czerwinski        0.039601
Sean P. Sullivan      0.038209
Anna Lee C. Iijima    0.033969
Jim Gordon            0.032138
Anne KrebiehlÂ MW     0.028352
Lauren Buzzeo         0.014119
Susan Kostrzewa       0.008348
Mike DeSimone         0.003955
Jeff Jenssen          0.003778
Alexander Peartree    0.003193
Carrie Dykes          0.001069
Fiona Adams           0.000208
Christina Pickard     0.000046
Name: taster_name, dtype: float64

In [22]:
df_nona = df[df['taster_name'].fillna('N') != 'N']

In [23]:
label_map = {val: idx for idx, val in enumerate(df['taster_name'].unique())}

In [24]:
y = df_nona['taster_name'].replace(label_map).values

In [25]:
reviews = df_nona.description

In [26]:
swords = set(stopwords.words('english'))

In [27]:
ps = PorterStemmer()

In [28]:
word_vecs = reviews\
    .apply(lambda x: [re.sub("[\W]", '', i).lower().strip() for i in x.split()])\
    .apply(lambda x: [ps.stem(i) for i in x if i not in swords and len(i) > 3])

In [29]:
word_vecs.sample(10)

18731     [gremillet, prestig, cuvã, made, chardonnay, b...
95138     [slight, cloudi, signal, natur, wine, smell, f...
17971     [opengrain, aroma, front, follow, sweeter, sce...
108587    [even, best, txakoli, wont, appeal, everyon, r...
62381     [2006, chilean, chardonnay, past, prime, mont,...
53595     [alway, loio, bright, fruiti, wine, readi, dri...
96541     [smell, full, also, soupi, stew, herbal, textu...
46031     [occasion, produc, reservelevel, wine, mysteri...
81070     [show, charact, upfront, start, aroma, cedar, ...
3592      [reach, matur, wine, show, tannin, soften, spi...
Name: description, dtype: object

In [30]:
back2sent = word_vecs.apply(lambda x: ' '.join(x))

In [31]:
transform = TfidfVectorizer(lowercase=False, min_df=.05)

In [32]:
tf_idf_matrix = transform.fit_transform(back2sent.values)

Data Preparation 2 (5pts)
Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

For our models, we used region_2, taster_name, variety.

Modeling and Evaluation 1 (10pts)
Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.

In [34]:
tf_idf_matrix.toarray()[:1000:100]

array([[0.19201608, 0.        , 0.34717677, 0.27458416, 0.18116137,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.29985157, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.65153631, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.16273315, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.29726321, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.268779  , 0.        , 0.18133404, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [35]:
from sklearn.naive_bayes import GaussianNB

In [36]:
test_train = np.random.random(word_vecs.shape[0]) < .8
Xtr = tf_idf_matrix[test_train]
Xte = tf_idf_matrix[~test_train]
ytr = y[test_train]
yte = y[~test_train]

In [37]:
model = GaussianNB()

In [38]:
model.fit(Xtr.toarray(), ytr)

GaussianNB(priors=None)

In [39]:
model.predict_proba(Xte.toarray()).shape

(20697, 19)

In [40]:
model.score(Xte.toarray(), yte)

0.4366333285017152

In [41]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
transform = TfidfVectorizer()

In [43]:
tf_idf_matrixtf_idf_m  = transform.fit_transform(back2sent.values)

Modeling and Evaluation 2 (10pts)

Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.

We used a train test split with a 20% hold out for model evaluation. Since we are using a multiclass classification our focus was on accuracy and not precision. 

In [44]:
## build train_test_split
test_train = np.random.random(word_vecs.shape[0]) < .8
Xtr = tf_idf_matrix[test_train]
Xte = tf_idf_matrix[~test_train]
ytr = y[test_train]
yte = y[~test_train]

Modeling and Evaluation 3 (20pts)
Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!

Gaussian Naiive Bayes Model

In [45]:
from sklearn.naive_bayes import GaussianNB

In [46]:
start = datetime.now()
modelgb = GaussianNB()
modelgb.fit(Xtr.toarray(), ytr)
print(modelgb.score(Xte.toarray(), yte))
print((datetime.now() - start).seconds)

0.3870296933939453
0


In [47]:
from sklearn.svm import SVC

In [48]:
start = datetime.now()
model_svm = SVC(C=1.0, kernel='rbf', degree=3, gamma=.05)
model_svm.fit(Xtr.toarray(), ytr)
print( model_svm.score(Xte.toarray(), yte))
print((datetime.now()-start).seconds)

0.7472192668536609
729


In [49]:
from sklearn.ensemble import RandomForestClassifier

In [52]:
## 50 Tree model
start = datetime.now()
modelrf100 = RandomForestClassifier(n_estimators=50, n_jobs=-1).fit(Xtr.toarray(), ytr)
print(RandomForestClassifier(n_estimators=50, n_jobs=-1).fit(Xtr.toarray(), ytr)\
    .score(Xte.toarray(), yte))
# modelrf100.fit(Xtr.toarray(), ytr)
# print(modelrf100.score(Xte.toarray(), yte))
print((datetime.now() - start).seconds)

0.7515717187348874
12


In [53]:
### 250 Tree model
start = datetime.now()
modelrf250 = RandomForestClassifier(n_estimators=250, n_jobs=-1)
modelrf250.fit(Xtr.toarray(), ytr)
print(modelrf250.score(Xte.toarray(), yte))
print((datetime.now() - start).seconds)

0.7569881032981913
32


In [54]:
### 1000 Tree Model
start = datetime.now()
modelrf1k = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
modelrf1k.fit(Xtr.toarray(), ytr)
print(modelrf1k.score(Xte.toarray(), yte))
print((datetime.now() - start).seconds)

0.7599380984621337
136


GradientBoosted Decision Trees Model

In [55]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
start = datetime.now()
modelgbf_t250 = GradientBoostingClassifier(max_depth=5,
                                           learning_rate=.1, 
                                           n_estimators=250)
modelgbf_t250.fit(Xtr.toarray(), ytr)
print(modelgbf_t250.score(Xte.toarray(), yte))
print((datetime.now() - start).seconds)

**Optimizing Document Frequency**

In [ ]:
reviews = ndf.description
y = ndf[y_col] == 'pinot noir'

swords = set(stopwords.words('english'))

ps = PorterStemmer()

word_vecs = reviews\
    .apply(lambda x: [re.sub("[\W]", '', i).lower().strip() for i in x.split()])\
    .apply(lambda x: [ps.stem(i) for i in x if i not in swords and len(i) > 3])

back2sent = word_vecs.apply(lambda x: ' '.join(x))

test_train = np.random.random(ndf.shape[0]) < .8
ytr = y[test_train]
yte = y[~test_train]

In [ ]:
scoresdf = {}
for key in scores.keys():
    scoresdf[key] = pd.DataFrame(scores[key])
    scoresdf[key].columns = ['min_df', 'time_taken', 'score']

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,8))
colors =[None, 'black', 'red', 'blue']
for key, vals in scoresdf.items():
    plt.plot(vals['min_df'], vals['score'], color=colors[key], label=key)
plt.legend(fontsize=18)

In [ ]:
scoresdf = {}
for key in scores.keys():
    scoresdf[key] = pd.DataFrame(scores[key])
    scoresdf[key].columns = ['min_df', 'time_taken', 'num_cols', 'score']

In [ ]:
scoresdf[1].head(20)

**Plot of Minimum Document Frequency vs Model Accuracy**
    - Colors represent n-gram size

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,8))
colors =[None, 'black', 'red', 'blue']
for key, vals in scoresdf.items():
    plt.plot(vals['min_df'], vals['score'], color=colors[key], label=key)
plt.legend(fontsize=18)

**Number of Features**


**Plot of number of total features vs Model accuracy**
    - Colors represent max number of n-grams allowed

In [ ]:
scoresdf_nfeats = {}
for key in feat_scores.keys():
    scoresdf_nfeats[key] = pd.DataFrame(feat_scores[key])
    scoresdf_nfeats[key].columns = ['max_features', 'time_taken', 'num_cols', 'score']
    
fig, ax = plt.subplots(1,1, figsize=(16,8))
colors =[None, 'black', 'red', 'blue']
for key, vals in scoresdf_nfeats.items():
    plt.plot(vals['max_features'], vals['score'], color=colors[key], label=key)
plt.legend(fontsize=18)

In [ ]:
print(scoresdf_nfeats[1]['time_taken']
      + scoresdf_nfeats[2]['time_taken'] +
      scoresdf_nfeats[3]['time_taken'])

**Number of Trees**


In [ ]:
scoresdf = pd.DataFrame(tree_scores)
scoresdf.columns = ['n_estimators', 'time_taken', 'num_cols', 'score']

**Plot of number of trees vs Model Accuracy**

In [ ]:
scoresdf.set_index('n_estimators')['score'].plot(figsize=(16,8))

**Max Tree Depth**

In [ ]:
depthdf = pd.DataFrame(depth_scores)
depthdf.columns = ['max_depth', 'time_taken', 'num_cols', 'score']

In [ ]:
depthdf.set_index('max_depth')['score'].plot(figsize=(16,8))

**Learning Rate**

In [ ]:
lr_scores_df = pd.DataFrame(lr_scores)
lr_scores_df.columns = ['learn_rate', 'time_taken', 'num_cols', 'score']

**Plot of Learning Rate vs Model Accuracy with 100 trees max depth of 10**

In [ ]:
lr_scores_df.set_index('learn_rate')['score'].plot(figsize=(16,8))

**Optimal model**

In [ ]:
label_map = {val: idx for idx, val in enumerate(ndf[y_col].unique())}
y = df_noblends[y_col].replace(label_map).values

In [ ]:
min_val = df_noblends['variety'].value_counts().tail(1).values[0]

print(min_val)
list_of_dfs = []
for x in df_noblends[y_col].unique():
    list_of_dfs.append(df_noblends[df_noblends[y_col] == x].sample(min_val))
ndf_all = pd.concat(list_of_dfs)

In [ ]:
ndf_all.shape

In [ ]:
reviews = ndf.description
swords = set(stopwords.words('english'))
ps = PorterStemmer()

word_vecs = reviews\
    .apply(lambda x: [re.sub("[\W]", '', i).lower().strip() for i in x.split()])\
    .apply(lambda x: [ps.stem(i) for i in x if i not in swords and len(i) > 3])

In [ ]:
back2sent = word_vecs.apply(lambda x: ' '.join(x))
transform = TfidfVectorizer(lowercase=False, max_df=.1, max_features=2000, ngram_range=(1,3))
tf_idf_matrix = transform.fit_transform(back2sent.values)

In [ ]:
tf_idf_matrix.toarray()[:1000:100]

In [ ]:
## Generate Cross validation set
rand_vals = np.random.random(word_vecs.shape[0])

cv_set = []
k= 5

for val in np.linspace(0,1-1/k,k):
    test_train = ((rand_vals <= val) | (rand_vals > val + 1.0/k))
    cv_set.append({'Xtr': tf_idf_matrix[test_train],
                    'Xte': tf_idf_matrix[~test_train],
                    'ytr': y[test_train],
                    'yte': y[~test_train]
                  })

In [ ]:
## Check Identified Optimal model parameters on Cross-validated set (k= 5)
mod_params = {'max_depth':25, 'learning_rate':.1, 'n_estimators' :3000, 'n_jobs': -1}
cv_scores = []

for tt_set in cv_set:
    cv_scores.append(xgb.XGBClassifier(**mod_params)\
                        .fit(tt_set['Xtr'].toarray(), tt_set['ytr'])\
                        .score(tt_set['Xte'].toarray(), tt_set['yte']))

In [ ]:
print(np.array(cv_scores).mean())

Modeling and Evaluation 5 (10pts)

Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.

We decided to go with XGBoost for our text classification because when using a 1000 estimators, XGBoost gave us ~56% while Random Forrest gave us 44%, when using 12 classes. 

Modeling and Evaluation 6 (10pts)

Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.

Since our final model had a total 2000 features and we performed ensemble modeling, it is difficult to accurately measure feature importance. Due to time constraints, computing 2000 features, would take about 250 days; if we left 1 in, 1 out. 

Deployment (5pts)

How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?

Based on our models, winerys and sommliers would be interested in using our analysis in order to help choose a fraud-prevention system. To this point, radio frequency identification technology is likely to be a major development in security devices. 

Exceptional Work (10pts)

You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm